In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Import the Data

In [ ]:
def load_data():
    
    print("Importing data, please wait...")
    import os
    import cv2
    
    root = r"/home/feketrob/Demosaiking/generated_training_data/"
    
    train_images = []
    train_truth = []
    test_images = []
    test_truth = []
    
    train_set = 0
    test_set = 0
    train_to_test = 0.8
    
    for folder in sorted(os.listdir(root)):
        
        #folder = "_t"
        path = os.path.join(root, folder)
        images = sorted(os.listdir(path))
        current_index = 0
        
        while current_index < len(images):
            image_paths = images[current_index: current_index + 3]
            image_b = cv2.imread(os.path.join(path, image_paths[0]), 0)
            image_g = cv2.imread(os.path.join(path, image_paths[1]), 0)
            image_r = cv2.imread(os.path.join(path, image_paths[2]), 0)
            image = np.zeros((image_b.shape[0], image_b.shape[1], 3))
            image = np.array([image_r, image_g, image_b])
            
            truth_paths = images[current_index + 3: current_index + 6]
            truth_b = cv2.imread(os.path.join(path, truth_paths[0]), 0)
            truth_g = cv2.imread(os.path.join(path, truth_paths[1]), 0)
            truth_r = cv2.imread(os.path.join(path, truth_paths[2]), 0)
            truth = np.zeros((truth_b.shape[0], truth_b.shape[1], 3))
            truth = np.array([truth_r, truth_g, truth_b])
            #truth = np.array([truth_b, truth_g, truth_r])
            
            if train_set > train_to_test * (train_set + test_set):
                test_images.append(image)
                test_truth.append(truth)
                test_set += 1
            else:
                train_images.append(image)
                train_truth.append(truth)
                train_set += 1
            
            current_index += 6
       
    print("Import finished. You can stop waiting now.")
    return ((np.asarray(train_images), np.asarray(train_truth)), (np.asarray(test_images), np.asarray(test_truth)))
    

In [ ]:
#import data

((train_images, train_truth), (test_images, test_truth)) = load_data()
print(train_images[0])
print(train_truth[0])
print(test_images[0])
print(test_truth[0])

Importing data, please wait...


## Explore the Data

In [ ]:
print('Training data shape : ', train_images.shape, train_truth.shape)

print('Testing data shape : ', test_images.shape, test_truth.shape)


## Preprocess the Data

In [ ]:
# Find the shape of input images and create the variable input_shape
nDims, nRows,nCols = train_images.shape[1:]
input_shape = (nRows, nCols, nDims)
train_data = train_images
test_data = test_images
train_data = np.array([image.swapaxes(0, 2).swapaxes(0, 1) for image in train_images])
test_data = np.array([image.swapaxes(0, 2).swapaxes(0, 1) for image in test_images])

print(train_data.shape)

nDims, nRows,nCols = train_truth.shape[1:]
train_truth_data = train_truth
test_truth_data = test_truth
train_truth_data = np.array([image.swapaxes(0, 2).swapaxes(0, 1) for image in train_truth])
test_truth_data = np.array([image.swapaxes(0, 2).swapaxes(0, 1) for image in test_truth])
print(train_truth_data.shape)

plt.figure(figsize=[4,2])


# Display the first image in training data
plt.subplot(121)
plt.imshow(train_data[0], cmap='gray')
plt.title("Input")

# Display the first image in testing data
plt.subplot(122)
plt.imshow(train_truth_data[0], cmap='gray')
plt.title("Result")


# Change to float datatype
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_truth_data = train_truth_data.astype('float32')
test_truth_data = test_truth_data.astype('float32')


print(train_truth_data[0])
# Scale the data to lie between 0 to 1
train_data /= 255
test_data /= 255
train_truth_data /= 255
test_truth_data /=255

def print_data(data):
    
    for line in data[0][:3]:
        for pixel in line[:3]:
            print(pixel)
    print("----")
    
print_data(train_data)
print_data(train_truth_data)
print_data(test_data)
print_data(test_truth_data)

## Define the model

In [ ]:
def gkern(kernlen=9, nsig=3):
    """Returns a 2D Gaussian kernel array."""

    import scipy.stats as st
    
    interval = (2*nsig+1.)/(kernlen)
    x = np.linspace(-nsig-interval/2., nsig+interval/2., kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
    kernel = kernel_raw/kernel_raw.sum()
    return kernel


def my_init(shape, dtype=None):
    
    print(shape)
    return gkern(shape[0])

def createModel():
    model = Sequential()
    
    model.add(Conv2D(128, (9, 9), activation='relu', input_shape=input_shape,
                    ))
    
    model.add(Conv2D(64, (1, 1), activation='relu'))
    model.add(Conv2D(3, (5, 5), activation='relu'))
    
    return model

## Load model

In [ ]:
def load_model():
    json_file = open(r'/home/feketrob/Demosaiking/models/long/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(r"/home/feketrob/Demosaiking/models/long/model.h5")
    print("Loaded model from disk")
    
    

## Train the model

In [ ]:
train = True
if train:
    model = createModel()
    batch_size = 256
    epochs = 10
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

    model.summary()

    history = model.fit(train_data, train_truth_data, batch_size=batch_size, epochs=epochs, verbose=1, 
                       validation_data=(test_data, test_truth_data))
    
else:
    from keras.models import model_from_json
    mode = load_model()
    model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

    
model.evaluate(test_data, test_truth_data)

## Check the loss and accuracy curves

In [ ]:
if train:
    plt.figure(figsize=[8,6])
    plt.plot(history.history['loss'],'r',linewidth=3.0)
    plt.plot(history.history['val_loss'],'b',linewidth=3.0)
    plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.title('Loss Curves',fontsize=16)

    plt.figure(figsize=[8,6])
    plt.plot(history.history['acc'],'r',linewidth=3.0)
    plt.plot(history.history['val_acc'],'b',linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Accuracy',fontsize=16)
    plt.title('Accuracy Curves',fontsize=16)

## Save model

In [ ]:
if train:
    # Save model
    import os
    import uuid

    models_rep = r"/home/feketrob/Demosaiking/models"
    folder_name = str(uuid.uuid1())
    target_folder = os.path.join(models_rep, folder_name)
    os.mkdir(target_folder)

    model_json = model.to_json()
    with open(os.path.join(target_folder, "model.json"), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(os.path.join(target_folder, "model.h5"))
    print("Saved model to disk")

    print(target_folder)

In [ ]:

inputf = test_data[0]

result = model.predict(np.array([inputf]))
inputf *= 255
result *= 255

inputf = inputf.astype("int32")
result = result.astype("int32")

print(inputf)
print(result)
plt.figure(figsize=[4,2])


# Display the first image in training data
plt.subplot(121)
plt.imshow(inputf, cmap='gray')
plt.title("Input")

# Display the first image in testing data
plt.subplot(122)
plt.imshow(result[0], cmap='gray')
plt.title("Result")